<a href="https://colab.research.google.com/github/eohta/udemy-numpyro-basic/blob/main/03_trips/01_fit_poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 【ポアソン分布】出張回数

出張回数のデータをポアソン分布にあてはめてみる。

## Package Installation

In [ ]:
!pip install numpyro

インストール完了後にランタイムを再スタートして下さい！

## Import Package

In [ ]:
import numpyro
import numpyro.distributions as dist

import jax
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
numpyro.set_platform('cpu')
numpyro.set_host_device_count(4)

In [ ]:
plt.rcParams['font.size'] = 12

## Load & Check Data

In [ ]:
y = np.array([
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 1, 0, 3, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0])

In [ ]:
print(y)

In [ ]:
np.mean(y)

In [ ]:
np.var(y)

In [ ]:
sns.histplot(y, discrete=True, shrink=0.7)

plt.xlim(-1, 5)
plt.xlabel('Number of Trips');

## Define Model & Inference

In [ ]:
def model(y=None, num_data=0):
    
    mu = numpyro.sample('mu', dist.HalfNormal(10))
    
    with numpyro.plate('data', num_data):
    
        numpyro.sample('obs', dist.Poisson(mu), obs=y)

In [ ]:
nuts = numpyro.infer.NUTS(model)
mcmc = numpyro.infer.MCMC(nuts, num_warmup=500, num_samples=3000, num_chains=4)

mcmc.run(jax.random.PRNGKey(0), y=y, num_data=len(y))
mcmc_samples = mcmc.get_samples()

idata = az.from_numpyro(mcmc)

## Check MCMC-samples

In [ ]:
az.plot_trace(idata)
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_posterior(idata);

## Posterior Predictive Check

In [ ]:
predictive = numpyro.infer.Predictive(model, mcmc_samples)

ppc_samples = predictive(jax.random.PRNGKey(1), num_data=len(y))

idata_ppc = az.from_numpyro(mcmc, posterior_predictive=ppc_samples)

In [ ]:
az.plot_ppc(idata_ppc, num_pp_samples=1000);

## Save Inference Data

In [ ]:
idata.to_netcdf('idata.nc')